In [1]:
Itemset = {
    1: 'Republican', 2: 'Democrat',
    3: 'handicapped-infants=Y', 4: 'handicapped-infants=N',
    5: 'water-project-cost-sharing=Y', 6: 'water-project-cost-sharing=N',
    7: 'adoption-of-the-budget-resolution=Y', 8: 'adoption-of-the-budget-resolution=N',
    9: 'physician-fee-freeze=Y', 10: 'physician-fee-freeze=N',
    11: 'el-salvador-aid=Y', 12: 'el-salvador-aid=N',
    13: 'religious-groups-in-schools=Y', 14: 'religious-groups-in-schools=N',
    15: 'anti-satellite-test-ban=Y', 16: 'anti-satellite-test-ban=N',
    17: 'aid-to-nicaraguan-contras=Y', 18: 'aid-to-nicaraguan-contras=N',
    19: 'mx-missile=Y', 20: 'mx-missile=N',
    21: 'immigration=Y', 22: 'immigration=N',
    23: 'synfuels-corporation-cutback=Y', 24: 'synfuels-corporation-cutback=N',
    25: 'education-spending=Y', 26: 'education-spending=N',
    27: 'superfund-right-to-sue=Y', 28: 'superfund-right-to-sue=N',
    29: 'crime=Y', 30: 'crime=N', 31: 'duty-free-exports=Y', 32: 'duty-free-exports=N',
    33: 'export-administration-act-south-africa=Y', 34: 'export-administration-act-south-africa=N'
}
SUP_min = 0.3
CONF_min = 0.9
#party = ''
#party = 'republic'
party = 'democrat'

# 加载数据，构建事务集
def loadData():
    f = open('data/house-votes-84.data')
    data = []
    for txt_line in f.read().split('\n'):
        data_processed = []
        if txt_line == '':
            pass
        else:
            tmp = txt_line.split(',')
            data_processed.append(1) if tmp[0] == 'republican' else data_processed.append(2)
            for i in range(len(tmp)):
                if tmp[i] == 'y':
                    data_processed.append(2 * i + 1)
                elif tmp[i] == 'n':
                    data_processed.append(2 * i + 2)
                else:
                    pass
            data.append(data_processed)
    f.close()
    return data


raw_data = loadData()

In [2]:
def split_data(raw_data, party):
    if party == '':
        return raw_data
    elif party == 'republic':
        data_republic = []
        for data in raw_data:
            if len(data) == 1:
                continue
            if data[0] == 1:
                data_republic.append(data[1:])
        return data_republic
    elif party == 'democrat':
        data_democrat = []
        for data in raw_data:
            if len(data) == 1:
                continue
            if data[0] == 2:
                data_democrat.append(data[1:])
        return data_democrat


data = split_data(raw_data, party=party)

In [3]:
# 挖掘 1-频繁集
def compare(item1, item2):
    tmp = []
    equal = True
    if len(item1) == 1:
        return equal, (item1 + item2)
    for i in range(len(item1) - 1):
        tmp.append(item1[i])
        if item1[i] != item2[i]:
            equal = False
            break
    if equal:
        minitem = min(item1[-1], item1[-1])
        maxitem = max(item2[-1], item1[-1])
        tmp.append(minitem)
        tmp.append(maxitem)
    return equal, tuple(tmp)


def issubset(item1, item2):
    # 判断 item1 是否为 item2 的子集
    return set(item1).issubset(set(item2))


def F1_Mining(data):
    C1, F1 = {}, {}
    count = len(data)
    for data_line in data:
        # 对于数据集中的每一行投票数据
        for i in range(len(data_line)):
            # 对于每一行数据中的下标（对应某个议题）
            key = tuple([data_line[i]])
            if key in C1.keys():
                # 以键值对的形式进行存储和计数
                C1[key] += 1
            else:
                C1[key] = 1
    for item in C1:
        if C1[item] / count >= SUP_min:
            F1[item] = C1[item]

    return F1


F1 = F1_Mining(data)

In [4]:
# 挖掘 2-频繁集
def F2_Mining(data, F1):
    C2, F2 = {}, {}
    F1_keys = sorted(list(F1.keys()))
    for i in range(len(F1_keys)):
        for j in range(i + 1, len(F1_keys)):
            equal, tmp = compare(F1_keys[i], F1_keys[j])
            if equal:
                C2[tmp] = 0
            else:
                continue
    for data_line in data:
        for item in C2:
            if issubset(item, data_line):
                C2[item] += 1
    for item in C2:
        if C2[item] / len(data) >= SUP_min:
            F2[item] = C2[item]
    return F2


F2 = F2_Mining(data, F1)

In [5]:
# 挖掘3-频繁项集乃至n-频繁项集，并合并所有频繁项集
def F_Mining(data, F1, F2):
    Fk_1 = F2
    F = [F1]  # 存储的是所有频繁项集
    while Fk_1 != {}:
        F.append(Fk_1)
        Ck, Fk = {}, {}
        Fk_1_keys = sorted(list(Fk_1.keys()))
        for i in range(len(Fk_1_keys)):
            for j in range(i + 1, len(Fk_1_keys)):
                equal, tmp = compare(Fk_1_keys[i], Fk_1_keys[j])
                if equal:
                    Ck[tmp] = 0
                else:
                    continue
        for data_line in data:
            for item in Ck:
                if issubset(item, data_line):
                    Ck[item] += 1
        for item in Ck:
            if Ck[item] / len(data) >= SUP_min:
                Fk[item] = Ck[item]
        Fk_1 = Fk
    return F


F = F_Mining(data, F1, F2)

In [6]:
def generate_1_rules(F):
    # 生成第一类关联规则
    rules = {}
    for i in range(2, len(F)):
        for item in F[i]:
            if 1 in item:
                antecedent = tuple(sorted(set(item) - {1}))  # 关联规则前件
                confidence = F[i][item] / F[i - 1][antecedent]
                if 1 > confidence >= CONF_min:
                    rules[(antecedent, 1)] = confidence
            elif 2 in item:
                antecedent = tuple(sorted(set(item) - {2}))  # 关联规则前件
                confidence = F[i][item] / F[i - 1][antecedent]
                if 1 > confidence >= CONF_min:
                    rules[(antecedent, 2)] = confidence
            else:
                continue
    return rules


rules = sorted(generate_1_rules(F).items(), key=lambda x: x[1], reverse=True)

In [7]:
def get_subset(freq):
    if len(freq) == 0:
        return [[]]
    all_subsets = get_subset(freq[1:]) + [[freq[0]] + r for r in get_subset(freq[1:])]
    all_subsets = sorted(all_subsets, key=lambda x: (len(x), x))
    return all_subsets


def subset_tuple(all_subsets, freq):
    # 整理子集
    all_subsets_tuple = []
    for subset in all_subsets:
        if len(subset) == 0 or len(subset) == len(freq):
            continue
        else:
            subset = set(subset)
            complement = set(list(freq)) - subset
            subset = tuple(sorted(subset, key=lambda x: x))
            complement = tuple(sorted(complement, key=lambda x: x))
            all_subsets_tuple.append(tuple([subset, complement]))
    return all_subsets_tuple


def generate_2_rules(F, party=party):
    # 生成第二类关联规则
    rules = {}
    for i in range(1, len(F)):
        for item in F[i]:
            for subset in subset_tuple(get_subset(item), item):
                rule_confidence = F[len(item) - 1][item] / F[len(subset[0]) - 1][subset[0]]
                if rule_confidence >= CONF_min:
                    rules[subset] = rule_confidence
    return rules


rules = sorted(generate_2_rules(F).items(), key=lambda x: x[1], reverse=True)

In [8]:
for rule in rules[:10]:
    antecedent = rule[0][0]
    consequent = rule[0][1]
    confidence = rule[1]
    antecedent_str = ''
    for item in antecedent:
        antecedent_str = antecedent_str + Itemset[item] + ', '
    print('关联规则:{:s} -> {:s}，置信度:{:.5f}'.format(antecedent_str, Itemset[consequent[0]], confidence))

关联规则:handicapped-infants=Y, synfuels-corporation-cutback=N,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:water-project-cost-sharing=N, el-salvador-aid=N,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:water-project-cost-sharing=N, anti-satellite-test-ban=Y,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:water-project-cost-sharing=N, aid-to-nicaraguan-contras=Y,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:water-project-cost-sharing=N, superfund-right-to-sue=N,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:water-project-cost-sharing=N, export-administration-act-south-africa=Y,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:el-salvador-aid=N, synfuels-corporation-cutback=N,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:religious-groups-in-schools=N, synfuels-corporation-cutback=N,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:anti-satellite-test-ban=Y, synfuels-corporation-cutback=N,  -> physician-fee-freeze=N，置信度:1.00000
关联规则:aid-to-nicaraguan-contras=Y, synfuels-corporation-cutback=N,  -> physician-fee